# MIV Feature Selection with Somers' D

This notebook demonstrates Marginal Information Value (MIV) based feature selection using rank correlation (Somers' D) instead of traditional WOE-based Information Value.


In [1]:
from pathlib import Path
from typing import Any

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from fastwoe import FastWoe
from fastwoe.display import StyledDataFrame
from fastwoe.modeling import gini_shapley, marginal_somersd_selection

## Load and Prepare Data

We'll use the Bank Case Study dataset to demonstrate MIV feature selection.


In [2]:
ROOT_DIR = Path.cwd().parent
df = pd.read_csv(ROOT_DIR / "data" / "BankCaseStudyData.csv")

label = "Final_Decision"
df[label] = df[label].map({"Accept": 0, "Decline": 1})

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list[Any](df.columns)}")
print("\nFirst few rows:")
df.head()

Dataset shape: (24859, 33)

Columns: ['Account_Number', 'Account_Type', 'Final_Decision', 'Current_Delinquency_status', 'Application_Date', 'Application_Score', 'Cheque_Card_Flag', 'Existing_Customer_Flag', 'Gross_Annual_Income', 'Home_Telephone_Number', 'Insurance_Required', 'Loan_Amount', 'Loan_Payment_Frequency', 'Loan_Payment_Method', 'Marital_Status', 'Number_of_Dependants', 'Number_of_Payments', 'Occupation_Code', 'Promotion_Type', 'Residential_Status', 'Time_at_Address', 'Time_in_Employment', 'Time_with_Bank', 'Weight_Factor', 'GB_Flag', 'Age_of_Applicant', 'Application_Month', 'Bureau_Score', 'SP_ER_Reference', 'SP_Number_Of_Searches_L6M', 'SP_Number_of_CCJs', 'Loan_to_income', 'split']

First few rows:


,Account_Number,Account_Type,Final_Decision,Current_Delinquency_status,Application_Date,Application_Score,Cheque_Card_Flag,Existing_Customer_Flag,Gross_Annual_Income,Home_Telephone_Number,...,Weight_Factor,GB_Flag,Age_of_Applicant,Application_Month,Bureau_Score,SP_ER_Reference,SP_Number_Of_Searches_L6M,SP_Number_of_CCJs,Loan_to_income,split
0,10730734532,FL,0,NaN,20061206,965,Y,Y,12000,N,...,2.0,NTU,28,200612,1009,2,0,0,15.50,Development
1,10803550208,VL,1,NaN,20060928,720,N,Y,10015,Y,...,2.0,Rejects,36,200609,784,3,0,0,29.96,Development
2,10769083290,FL,0,0.0,20060721,975,Y,N,11000,Y,...,2.0,Good,48,200607,940,1,2,0,45.45,Development
3,10072636331,FL,0,1.0,20060529,960,Y,N,16500,Y,...,2.0,Good,41,200605,902,1,1,0,31.82,Development
4,10737329597,FL,0,0.0,20060718,980,Y,Y,60000,Y,...,2.0,Good,37,200607,1013,1,7,0,16.67,Development


In [3]:
# Select categorical features for feature selection
categorical_features = [
    "Account_Type",
    "Cheque_Card_Flag",
    "Existing_Customer_Flag",
    "Home_Telephone_Number",
    "Insurance_Required",
    "Loan_Payment_Frequency",
    "Loan_Payment_Method",
    "Marital_Status",
    "Occupation_Code",
    "Promotion_Type",
    "Residential_Status",
]
numerical_features = [
    "Gross_Annual_Income",
    "Number_of_Dependants",
    "Time_at_Address",
    "Time_in_Employment",
    "Time_with_Bank",
    "Age_of_Applicant",
    # "Bureau_Score",
    "Application_Score",
    "SP_Number_Of_Searches_L6M",
    "SP_Number_of_CCJs",
    "Loan_to_income",
]

# Filter to features that exist and have reasonable cardinality
# available_features = [f for f in categorical_features if f in df.columns]
available_features = categorical_features + numerical_features
df[numerical_features] = df[numerical_features].fillna(df[numerical_features].median())

print(f"Available categorical features: {available_features}")

# Check cardinality
for feat in available_features:
    n_unique = df[feat].nunique()
    print(f"{feat}: {n_unique} unique values")

Available categorical features: ['Account_Type', 'Cheque_Card_Flag', 'Existing_Customer_Flag', 'Home_Telephone_Number', 'Insurance_Required', 'Loan_Payment_Frequency', 'Loan_Payment_Method', 'Marital_Status', 'Occupation_Code', 'Promotion_Type', 'Residential_Status', 'Gross_Annual_Income', 'Number_of_Dependants', 'Time_at_Address', 'Time_in_Employment', 'Time_with_Bank', 'Age_of_Applicant', 'Application_Score', 'SP_Number_Of_Searches_L6M', 'SP_Number_of_CCJs', 'Loan_to_income']
Account_Type: 2 unique values
Cheque_Card_Flag: 2 unique values
Existing_Customer_Flag: 2 unique values
Home_Telephone_Number: 2 unique values
Insurance_Required: 2 unique values
Loan_Payment_Frequency: 4 unique values
Loan_Payment_Method: 4 unique values
Marital_Status: 5 unique values
Occupation_Code: 4 unique values
Promotion_Type: 4 unique values
Residential_Status: 4 unique values
Gross_Annual_Income: 2316 unique values
Number_of_Dependants: 12 unique values
Time_at_Address: 479 unique values
Time_in_Employ

In [4]:
# Prepare feature matrix (only categorical features for now)
X = df[available_features].copy()
y = df[label].values

# Remove rows with too many missing values
missing_threshold = 0.5
valid_mask = (X.isna().sum(axis=1) / len(X.columns)) < missing_threshold
X = X[valid_mask].copy()
y = y[valid_mask]

print(f"Final dataset shape: {X.shape}")
print(f"Target distribution: {pd.Series(y).value_counts().to_dict()}")
print(f"Bad rate: {y.mean():.2%}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"\nTrain: {len(X_train)}, Test: {len(X_test)}")

Final dataset shape: (24859, 21)
Target distribution: {0: 22262, 1: 2597}
Bad rate: 10.45%

Train: 17401, Test: 7458


## Run MIV Feature Selection

Now we'll run the MIV feature selection using Somers' D rank correlation.


In [5]:
# Run MIV feature selection
result = marginal_somersd_selection(
    X_train,
    y_train,
    X_test=X_test,
    y_test=y_test,
    min_miv=0.01,  # Minimum marginal Somers' D threshold
    max_features=10,  # Maximum number of features to select
    correlation_threshold=0.6,  # Maximum correlation between features
    ties="y",
    random_state=42,
)

Pre-computing WOE values for all features...
Building pairwise feature correlation matrix using Gini pairwise...
Calculating univariate Somers' D for all features...

Step 1: Selected 'Time_with_Bank' (Somers' D: 0.6376)
Step 2: Selected 'SP_Number_of_CCJs' (Marginal Somers' D: 0.5746)
Step 3: Selected 'Existing_Customer_Flag' (Marginal Somers' D: 0.4920)
Step 4: Selected 'Home_Telephone_Number' (Marginal Somers' D: 0.4948)
Step 5: Selected 'Gross_Annual_Income' (Marginal Somers' D: 0.4200)
Step 6: Selected 'Application_Score' (Marginal Somers' D: 0.5749)
Step 7: Selected 'Account_Type' (Marginal Somers' D: 0.3345)
Step 8: Selected 'Residential_Status' (Marginal Somers' D: 0.3094)
Step 9: Selected 'Loan_to_income' (Marginal Somers' D: 0.2714)
Step 10: Selected 'Cheque_Card_Flag' (Marginal Somers' D: 0.3125)

Reached max_features limit (10)


## Using the Final Model with Selected Features

The feature selection returns a trained FastWoe model that can be used directly for predictions. Here's how to use it:


In [6]:
final_model = result["model"]
selected_features = result["selected_features"]

print(f"Selected {len(selected_features)} features:")
for i, feat in enumerate[Any](selected_features, 1):
    print(f"{i}. {feat}")

# The model is already trained and ready to use
print(f"\nModel is fitted: {final_model.is_fitted_}")
# FastWoe stores features in mappings_ dictionary
print(f"Model features: {list[Any](final_model.mappings_.keys())}")

Selected 10 features:
1. Time_with_Bank
2. SP_Number_of_CCJs
3. Existing_Customer_Flag
4. Home_Telephone_Number
5. Gross_Annual_Income
6. Application_Score
7. Account_Type
8. Residential_Status
9. Loan_to_income
10. Cheque_Card_Flag

Model is fitted: True
Model features: ['Time_with_Bank', 'SP_Number_of_CCJs', 'Existing_Customer_Flag', 'Home_Telephone_Number', 'Gross_Annual_Income', 'Application_Score', 'Account_Type', 'Residential_Status', 'Loan_to_income', 'Cheque_Card_Flag']


## Adding Custom Features to the Model

You can add custom features to the selected model by:

1. Creating a new FastWoe model with additional features
2. Using a custom FastWoe configuration
3. Manually selecting features to include


In [7]:
# Create a custom FastWoe model with specific parameters
custom_woe_config = FastWoe(
    binning_method="faiss_kmeans",
    random_state=42,
)

# Use it in feature selection
result_custom = marginal_somersd_selection(
    X_train,
    y_train,
    X_test=X_test,
    y_test=y_test,
    min_miv=0.01,
    max_features=10,
    correlation_threshold=0.6,
    ties="y",
    random_state=42,
    woe_model=custom_woe_config,  # Pass custom configuration
)

print(f"Custom config selected {len(result_custom['selected_features'])} features")
print(f"Features: {result_custom['selected_features']}")

Pre-computing WOE values for all features...
Building pairwise feature correlation matrix using Gini pairwise...
Calculating univariate Somers' D for all features...

Step 1: Selected 'Application_Score' (Somers' D: 0.5048)
Step 2: Selected 'Occupation_Code' (Marginal Somers' D: 0.4998)
Step 3: Selected 'SP_Number_of_CCJs' (Marginal Somers' D: 0.4396)
Step 4: Selected 'Loan_to_income' (Marginal Somers' D: 0.3760)
Step 5: Selected 'Existing_Customer_Flag' (Marginal Somers' D: 0.3357)
Step 6: Selected 'SP_Number_Of_Searches_L6M' (Marginal Somers' D: 0.3352)
Step 7: Selected 'Cheque_Card_Flag' (Marginal Somers' D: 0.3750)
Step 8: Selected 'Home_Telephone_Number' (Marginal Somers' D: 0.3741)
Step 9: Selected 'Residential_Status' (Marginal Somers' D: 0.2670)
Step 10: Selected 'Account_Type' (Marginal Somers' D: 0.3074)

Reached max_features limit (10)
Custom config selected 10 features
Features: ['Application_Score', 'Occupation_Code', 'SP_Number_of_CCJs', 'Loan_to_income', 'Existing_Custom

## Shapley Value Decomposition for Gini Contributions (Linear SHAP)

Shapley values provide a fair attribution of Gini contributions using a **closed-form Linear SHAP solution**. When a base score is specified, the function:

- Fixes the population to where the base score is available
- Always includes the base score in the averaged score
- Computes Shapley values only for "extras" (scores other than base)
- Shows base-only Gini separately and incremental effects for extras


In [8]:
# 1. Internal model score (from our selected features)
internal_score = final_model.predict_proba(X_test[selected_features])[:, 1]

n = len(internal_score)
y_test = np.asarray(y_test)

rng = np.random.default_rng(42)

# 2. External bureau score (independent source)
bureau_score = rng.uniform(0.2, 0.8, n)
bureau_score = 0.7 * bureau_score + 0.3 * internal_score + rng.normal(0, 0.1, n)
bureau_score = np.clip(bureau_score, 0, 1)

has_bureau = np.ones(n, dtype=bool)  # usually available

# 3. Internal behavioral score (existing customers only)
if "Existing_Customer_Flag" in X_test.columns:
    existing_mask = X_test["Existing_Customer_Flag"] == "Y"
else:
    # simulate existing customers
    existing_mask = rng.random(n) < 0.5

behavioral_score = np.zeros(n)

# Behavioral score depends on internal score + noise
behavioral_score[existing_mask] = (
    0.6 * internal_score[existing_mask]
    + 0.4 * rng.uniform(0.3, 0.9, existing_mask.sum())
    + rng.normal(0, 0.1, existing_mask.sum())
)
behavioral_score = np.clip(behavioral_score, 0, 1)

has_behavioral = existing_mask.copy()

# 4. Alternative model score
if alt_features := [f for f in X_train.columns if f not in selected_features][:5]:
    alt_model = FastWoe()
    alt_model.fit(X_train[alt_features], y_train)
    alternative_score = alt_model.predict_proba(X_test[alt_features])[:, 1]
else:
    # fallback: weak independent signal
    alternative_score = (
        0.5 * rng.uniform(0.25, 0.75, n) + 0.5 * internal_score + rng.normal(0, 0.1, n)
    )
    alternative_score = np.clip(alternative_score, 0, 1)

has_alternative = np.ones(n, dtype=bool)

# Assemble score dictionary
score_dict = {
    "Internal_Model": internal_score,
    "Bureau_Score": bureau_score,
    "Behavioral_Score": behavioral_score,
    "Alternative_Model": alternative_score,
}

availability_mask = {
    "Internal_Model": np.ones(n, dtype=bool),
    "Bureau_Score": has_bureau,
    "Behavioral_Score": has_behavioral,
    "Alternative_Model": has_alternative,
}

# Compute Shapley values with base score decomposition
# This uses the closed-form Linear SHAP solution with a base score
shapley_df = gini_shapley(
    score_dict=score_dict,
    y=y_test,
    availability_mask=availability_mask,
    base_score_name="Internal_Model",  # Base score is always included
    ties="y",
)

StyledDataFrame(
    shapley_df,
    title="Final System Gini Decomposition (Shapley, averaged score)",
    subtitle=(
        "Base-only Gini is shown for the internal model alone. "
        "Incremental effects for other scores/rules are Shapley values under the production averaging rule "
        "and sum (approximately) to final_system_gini - base_only_gini. "
        "Availability is respected per component; population is fixed to base-available samples."
    ),
    highlight_cols=[
        "effect_on_gini",
        "base_only_gini",
        "final_system_gini",
        "base_plus_incrementals",
    ],
    precision=4,
)

component,effect_on_gini,role,base_only_gini,final_system_gini,sum_incremental_effects,base_plus_incrementals,n_samples_used,n_pos,n_neg
Internal_Model,0.4504,base_score_only,0.4504,0.5411,0.0907,0.5411,7458,779,6679
Alternative_Model,0.1022,increment_over_base,0.4504,0.5411,0.0907,0.5411,7458,779,6679
Behavioral_Score,-0.0011,increment_over_base,0.4504,0.5411,0.0907,0.5411,7458,779,6679
Bureau_Score,-0.0103,increment_over_base,0.4504,0.5411,0.0907,0.5411,7458,779,6679
